In [1]:
import torch

print(torch.backends.mps.is_available())

print(torch.backends.mps.is_built())

True
True


In [20]:
from func.utils import read_data
from func.utils import save_processed_csv

In [3]:
df = read_data('sia')
df

,textID,text,selected_text,sentiment,neg,neu,pos,compound
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0.000,1.000,0.000,0.0000
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,0.474,0.526,0.000,-0.7437
2,088c60f138,my boss is bullying me...,bullying me,negative,0.494,0.506,0.000,-0.5994
3,9642c003ef,what interview! leave me alone,leave me alone,negative,0.538,0.462,0.000,-0.3595
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,0.128,0.722,0.150,0.1027
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,0.000,0.890,0.110,0.3818
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,0.000,0.572,0.428,0.9136
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,0.000,0.680,0.320,0.3291


# Roberta Pretrained Model

In [4]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

import pandas as pd
import numpy as np 

from tqdm.notebook import tqdm

In [5]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [6]:
example = df.sample(1)['text'].values[0]
example

' I just retweeted your jesus post as it made me laugh... don`t worry, wasn`t a subliminal code'

In [7]:
encoded_text = tokenizer(example,return_tensors='pt')

In [8]:
output = model(**encoded_text)
scores = output[0][0].detach().numpy()

In [9]:
scores = softmax(scores)
score_dict = {
    'roberta_neg':scores[0],
    'roberta_neu':scores[1],
    'roberta_pos':scores[2]
}

print(example)
print(score_dict)

 I just retweeted your jesus post as it made me laugh... don`t worry, wasn`t a subliminal code
{'roberta_neg': 0.007064632, 'roberta_neu': 0.16795412, 'roberta_pos': 0.8249812}


## Function for Roberta 

In [10]:
def roberta_polatity(text):
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    score_dict = {
        'roberta_neg':scores[0],
        'roberta_neu':scores[1],
        'roberta_pos':scores[2]
    }
    return score_dict


In [11]:
# Getting the polarity of which row in the dataframe

res = dict()

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = str(row['text'])
        id = row['textID']
        res[id] = roberta_polatity(text)
    except(KeyboardInterrupt):
        break
    except:
        print(id)
        
    

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/27481 [00:00<?, ?it/s]

In [18]:
df_roberta = pd.DataFrame(res).T.reset_index().rename(columns={'index':'textID'})
df = df.merge(df_roberta)
df

,textID,text,selected_text,sentiment,neg,neu,pos,compound,roberta_neg,roberta_neu,roberta_pos
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0.000,1.000,0.000,0.0000,0.064939,0.808318,0.126744
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,0.474,0.526,0.000,-0.7437,0.918159,0.066099,0.015742
2,088c60f138,my boss is bullying me...,bullying me,negative,0.494,0.506,0.000,-0.5994,0.924613,0.070741,0.004646
3,9642c003ef,what interview! leave me alone,leave me alone,negative,0.538,0.462,0.000,-0.3595,0.783082,0.192980,0.023938
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,0.000,1.000,0.000,0.0000,0.564197,0.404575,0.031229
...,...,...,...,...,...,...,...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative,0.128,0.722,0.150,0.1027,0.434403,0.445122,0.120474
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative,0.000,0.890,0.110,0.3818,0.139543,0.635025,0.225433
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive,0.000,0.572,0.428,0.9136,0.003337,0.022629,0.974034
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive,0.000,0.680,0.320,0.3291,0.053331,0.357757,0.588912


In [21]:
save_processed_csv(df, 'twitter', 'df_sia_roberta')